In [3]:
import yfinance as yf
import numpy as np
import pandas as pd

In [69]:

#Key Metrics คือปัจจัยหรือตัวชี้วัดที่สำคัญที่สามารถบ่งบอกได้ว่าสิ่งใดจะทำให้บริษัทประสบความ
# สำเร็จหรือไม่สำเร็จ และมีโอกาสที่จะเติบโตในอนาคตหรือไม่

# returns คือผลตอบแทนรายปี มาจาก   (ราคาปิดวันสุดท้ายของปี/ราคาปิดวันแรกของปี - 1 ) *100
# volatility คือความผันผวนราคาหุ้นในปีนั้น มาจาก std ของผลรวมราคาปิดหุ้นรายวัน(yfinance)
# earningPerShare กำไรสุทธิต่อหุ้น (กำไรต่อหุ้น (บาท) set.or)
# currentRatio อัตราส่วนความสามารถในการชำระหนี้ (investing-งบดุล-สินทรัพย์สภาพคล่องรวม/หนี้สินหมุนเวียนรวม) งบดุล
# debtToAssets  หรือ L/A อัตราส่วนหนี้รวมต่อสินทรัพย์รวม  ยิ่งต่ำยิ่งดี เพราะแสดงว่าบริษัทมีปริมาณหนี้สินที่น้อย หรืออาจกล่าวได้ว่า มีความสามารถในการชำระหนี้สูง (investing-งบดุล-สินทรัพย์รวม/หนี้สินรวม) งบดุล
# debtToEquity คือ อัตราส่วนที่บอกว่า บริษัทมีหนี้สินเป็นกี่เท่าของส่วนของผู้ถือหุ้น (investing งบดุล-สินทรัพ/ส่วนผู้ถือหุ้น)
# dividendYeild คือ อัตราส่วนระหว่างเงินปันผลต่อหุ้นและราคาหุ้น วัดออกมาเป็น% เพื่อคำนวณความคุ้มค่าก่อนลงทุน (อัตราส่วนเงินปันผลตอบแทน (%) set.or)
# earningsYield คือ eps(กำไรสุทธิต่อหุ้น) หารด้วย ราคาหุ้นในตอนนั้น (set.or )

#investing
#หน้างบดุล

# set.or
#eps กำไรสุทธิต่อหุ้น
#dividendYeild อัตราส่วนเงินปันผลตอบแทน
#earningsYield = eps/ราคาหุ้นในตอนนั้น

#จัดอันดับจากcluster จากสูตร Sharpe Ratio คือ อัตราส่วนผลตอบแทนส่วนเกินต่อส่วนเบี่ยงเบนมาตรฐาน(ความผันผวน)  มันคุ้มค่าไหมกับที่เสี่ยงไป เลขยิ่งสูงยิ่งดี เลขต่ำคือไม่ดี
#['returnsTopricevol'] = ['returns'] / ['volatility'] 


# สรุปวิธีการคร่าวๆ 
#ในการเตรียม dataset จะมีปี 63 64 65 จะจัดกลุ่มหุ้นปี 63 ไปวิเคราะห์ปี64 แล้วก็จัดกลุ่มหุ้นปี64 ไปวิเคราะห์ปี65 ด้วย AgglomerativeClustering 
#ใช้หุ้นที่ได้รับคัดเลือกจาก cluster แต่ละปีจะนำไปทดสอบใน dataset ของปีถัดไป
#.dot ผลตอบแทนจากหุ้นที่เราคัดมาในปีถัดไปมาคุณscalarกับจำนวนหุ้นในแต่ละคลัสเตอร์ จากนั้นหารด้วยจำนวนหุ้น จะได้ค่าเฉลี่ยถ่วงน้ำหนักของผลตอบแทน
#returns_cluster_18_17 = perform_18_cluster_17['returns'].dot(perform_18_cluster_17['num_of_stocks']) / perform_18_cluster_17['num_of_stocks'].sum()

#คืออัลกอริธึมการจัดกลุ่มแบบลำดับชั้นประเภทหนึ่ง เป็นเทคนิคแมชชีนเลิร์นนิงแบบunsupervisedที่แบ่งประชากรออกเป็นหลายคลัสเตอร์ เพื่อให้จุดข้อมูลในคลัสเตอร์เดียวกันมีความคล้ายคลึงกันมากขึ้น
#จะทดลอง2แบบ แบบแรกจะเลือก3กลุ่มแรก แบบที่สองจะเลือกtop10หุ้นที่มีอัตราส่วนผลตอบแทนต่อความผันผวนสูง(['returns'] / ['volatility'])โดยได้จากการจัดอันดับเฉยๆไม่ได้ใช้clusteringมาช่วย แล้วนำมาทดสอบในปีถัดๆไป

#z-score
#ข้อเสีย 1.การประเมินหุ้นใช้คำนวณจากราคาวันแรกและราคาในวันสุดท้ายของปี ซึ่งความเป็นจริงถ้าเรานำไปประยุกต์ใช้จริงอาจจะหาจังหวะในการเข้าซื้อที่ราคาเหมาะสมกว่านี้ได้
#ปัญหา yfinance 1.ข้อมูลหลายส่วนไม่มีข้อมูลย้อนหลังหรือไม่มีข้อมูลให้เลย
# EV / EBITDA สามารถสูงกว่าผลตอบแทนจากสินทรัพย์ได้เสมอ ดังนั้นเพื่อป้องกันไม่ให้ความแตกต่างของมาตราส่วนดังกล่าวนำไปสู่น้ำหนักที่ไม่มีใครเทียบได้และข้อสรุปที่ไม่น่าเชื่อถือ เราต้องทำให้ข้อมูลเป็นมาตรฐานก่อน

In [4]:
nof = {'num_of_stocks' : [1,6,3] , 'returns':[100.107,14.449,40.518]}
 

tdf = pd.DataFrame(nof)

tdfval = tdf['returns'].dot(tdf['num_of_stocks']) / tdf['num_of_stocks'].sum()
print(tdfval)



30.835500000000003


In [5]:
set50_key = ['ADVANC','AOT','AWC','BANPU','BBL','BDMS','BEM','BGRIM','BH','BLA','BTS','CBG','CPALL',
           'CPF','CPN','CRC','DTAC','EA','EGCO','GLOBAL','GPSC','GULF','HMPRO','INTUCH','IRPC','IVL',
           'JMART','JMT','KBANK','KCE','KTB','KTC','LH','MINT','MTC','OR','PTT','PTTEP','PTTGC','SAWAD'
           'SCB','SCC','SCGP','TIDLOR','TISCO','TOP','TRUE','TTB','TU']

In [6]:
import yfinance as yf
import numpy as np
import pandas as pd
from pandas_datareader import data as pdr
yf.pdr_override()
#aapl_historical = aapl.history(start="2020-06-02", end="2020-06-07", interval="1m")
stock = yf.Ticker('ADVANC.BK')
stock2 = yf.Ticker('KBANK.BK')
df = stock.history(period='2y')
df.tail()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2022-11-15 00:00:00+07:00,189.0,189.0,187.0,187.5,5067300,0.0,0
2022-11-16 00:00:00+07:00,187.0,189.0,187.0,187.5,5616900,0.0,0
2022-11-17 00:00:00+07:00,187.5,187.5,186.0,187.0,5787400,0.0,0
2022-11-18 00:00:00+07:00,188.0,190.0,187.5,190.0,7921400,0.0,0
2022-11-21 00:00:00+07:00,189.5,191.0,188.5,190.5,4997503,0.0,0


In [ ]:
data = yf.download("AMZN AAPL GOOG", start="2017-01-01", end="2017-04-30")
data.Close

In [187]:
stock.dividends

Date
2021-02-19 00:00:00+07:00    3.68
2021-08-16 00:00:00+07:00    3.45
2022-02-18 00:00:00+07:00    4.24
2022-08-19 00:00:00+07:00    3.45
Name: Dividends, dtype: float64

In [75]:
#'forwardEps' 
stock.earnings_history

,Symbol,Company,Earnings Date,EPS Estimate,Reported EPS,Surprise(%)
0,ADVANC.BK,Advanced Info Service PCL,"Nov 01, 2023, 5 PMICT",NaN,NaN,NaN
1,ADVANC.BK,Advanced Info Service PCL,"Aug 07, 2023, 4 PMICT",NaN,NaN,NaN
2,ADVANC.BK,Advanced Info Service PCL,"May 08, 2023, 4 PMICT",NaN,NaN,NaN
3,ADVANC.BK,Advanced Info Service PCL,"Feb 06, 2023, 4 PMICT",2.43,NaN,NaN
4,ADVANC.BK,Advanced Info Service PCL,"Nov 04, 2022, 5 PMICT",2.13,2.03,-4.69
5,ADVANC.BK,Advanced Info Service PCL,"Nov 03, 2022, 5 PMICT",2.13,2.03,-4.69
6,ADVANC.BK,Advanced Info Service PCL,"Aug 17, 2022, 5 PMICT",2.27,2.12,-6.61
7,ADVANC.BK,Advanced Info Service PCL,"Aug 09, 2022, 8 AMICT",2.27,2.12,-6.61
8,ADVANC.BK,Advanced Info Service PCL,"Aug 08, 2022, 4 PMICT",2.27,2.12,-6.61


In [76]:
stock.dividends

Date
2021-02-19 00:00:00+07:00    3.68
2021-08-16 00:00:00+07:00    3.45
2022-02-18 00:00:00+07:00    4.24
2022-08-19 00:00:00+07:00    3.45
Name: Dividends, dtype: float64

In [ ]:
stock.info

In [9]:
stock.cashflow

,2021-12-31,2020-12-31,2019-12-31,2018-12-31
Investments,-2.999999e+08,-2.468998e+08,-8.056000e+06,-7.880758e+08
Change To Liabilities,7.355781e+08,-9.024916e+08,2.130084e+09,1.898272e+09
Total Cashflows From Investing Activities,-4.535194e+10,-5.469445e+10,-2.678435e+10,-4.387675e+10
Net Borrowings,-2.206707e+10,-7.682170e+09,-1.510981e+10,-7.936396e+08
Total Cash From Financing Activities,-4.696317e+10,-3.212388e+10,-3.928849e+10,-2.683242e+10
Change To Operating Activities,-6.863694e+08,-1.616951e+08,6.447462e+08,-1.065462e+08
Net Income,2.692215e+10,2.743436e+10,3.118957e+10,2.968218e+10
Change In Cash,-5.681479e+09,-1.215859e+09,1.056974e+10,-1.583519e+09
Effect Of Exchange Rate,-4.155650e+05,-2.692110e+07,1.533469e+07,-6.042459e+06
Total Cash From Operating Activities,8.663404e+10,8.562939e+10,7.662724e+10,6.913169e+10


In [7]:
stock.get_financials()

,2021-12-31,2020-12-31,2019-12-31,2018-12-31
Research Development,None,None,None,None
Effect Of Accounting Charges,None,None,None,None
Income Before Tax,32894011689.0,32525841133.0,37401761871.0,35636643874.0
Minority Interest,125770045.0,125383375.0,128070516.0,147540956.0
Net Income,26922145655.0,27434360343.0,31189570687.0,29682178137.0
Selling General Administrative,21845937713.0,24498658961.0,27884915113.0,26472313353.0
Gross Profit,59879696530.0,62041942495.0,68994240425.0,66400158364.0
Ebit,38033758817.0,37543283534.0,41109325312.0,39927845011.0
Operating Income,38033758817.0,37543283534.0,41109325312.0,39927845011.0
Other Operating Expenses,None,None,None,None


In [8]:
stock.balancesheet

,2021-12-31,2020-12-31,2019-12-31,2018-12-31
Intangible Assets,1.426390e+11,1.173421e+11,1.085424e+11,1.168409e+11
Capital Surplus,2.250630e+10,2.244653e+10,2.238607e+10,2.237228e+10
Total Liab,2.743972e+11,2.744812e+11,2.202750e+11,2.328364e+11
Total Stockholder Equity,8.169875e+10,7.556398e+10,6.926610e+10,5.752104e+10
Minority Interest,1.257700e+08,1.253834e+08,1.280705e+08,1.475410e+08
Other Current Liab,2.017737e+10,2.025782e+10,3.478309e+10,1.360204e+10
Total Assets,3.562217e+11,3.501706e+11,2.896691e+11,2.905050e+11
Common Stock,2.973926e+09,2.973554e+09,2.973179e+09,2.973095e+09
Other Current Assets,2.345092e+09,2.131939e+09,2.193347e+09,2.656159e+09
Retained Earnings,5.710252e+10,5.138243e+10,4.422558e+10,3.250511e+10


In [193]:
stock.dividends

Date
2021-02-19 00:00:00+07:00    3.68
2021-08-16 00:00:00+07:00    3.45
2022-02-18 00:00:00+07:00    4.24
2022-08-19 00:00:00+07:00    3.45
Name: Dividends, dtype: float64

In [11]:
set100_key = ['ADVANC','AOT','AWC','BANPU','BBL','BDMS','BEM','BGRIM','BH','BLA','BTS','CBG','CPALL',
           'CPF','CPN','CRC','DTAC','EA','EGCO','GLOBAL','GPSC','GULF','HMPRO','INTUCH','IRPC','IVL',
           'JMART','JMT','KBANK','KCE','KTB','KTC','LH','MINT','MTC','OR','OSP','PTT','PTTEP','PTTGC','SAWAD',
           'SCB','SCC','SCGP','TIDLOR','TISCO','TOP','TRUE','TTB','TU','ACE','AEONTS','AMATA','AP','BAM','BCP',
           'BCPG','BEC','CENTEL','CHG','CK','CKP','COM7','DOHOME','EPG','ESSO','FORTH','GUNKUL','HANA',
           'KEX','KKP','MAJOR','MEGA','ONEE','ORI','PLANB','PSL','PTG','QH','RATCH','RBF','RCL','SINGER',
           'SPALI','SPRC','STA','STARK','STEC','STGT','SUPER','SYNEX','TASCO','TCAP','THANI','TIPH','TQM',
           'TTA','VGI','WHA','BCH']

# liststock = [yf.Ticker(x+'.BK') for x in set50_key]
liststock = [(x+'.BK') for x in set100_key]


In [12]:
print(len(liststock))

100


In [25]:
import pandas as pd


,2021-12-31,2020-12-31,2019-12-31,2018-12-31
Intangible Assets,1.426390e+11,1.173421e+11,1.085424e+11,1.168409e+11
Capital Surplus,2.250630e+10,2.244653e+10,2.238607e+10,2.237228e+10
Total Liab,2.743972e+11,2.744812e+11,2.202750e+11,2.328364e+11
Total Stockholder Equity,8.169875e+10,7.556398e+10,6.926610e+10,5.752104e+10
Minority Interest,1.257700e+08,1.253834e+08,1.280705e+08,1.475410e+08
Other Current Liab,2.017737e+10,2.025782e+10,3.478309e+10,1.360204e+10
Total Assets,3.562217e+11,3.501706e+11,2.896691e+11,2.905050e+11
Common Stock,2.973926e+09,2.973554e+09,2.973179e+09,2.973095e+09
Other Current Assets,2.345092e+09,2.131939e+09,2.193347e+09,2.656159e+09
Retained Earnings,5.710252e+10,5.138243e+10,4.422558e+10,3.250511e+10


In [64]:
yf.Ticker('ADVANC.BK').balancesheet



,2021-12-31,2020-12-31,2019-12-31,2018-12-31
Intangible Assets,1.426390e+11,1.173421e+11,1.085424e+11,1.168409e+11
Capital Surplus,2.250630e+10,2.244653e+10,2.238607e+10,2.237228e+10
Total Liab,2.743972e+11,2.744812e+11,2.202750e+11,2.328364e+11
Total Stockholder Equity,8.169875e+10,7.556398e+10,6.926610e+10,5.752104e+10
Minority Interest,1.257700e+08,1.253834e+08,1.280705e+08,1.475410e+08
Other Current Liab,2.017737e+10,2.025782e+10,3.478309e+10,1.360204e+10
Total Assets,3.562217e+11,3.501706e+11,2.896691e+11,2.905050e+11
Common Stock,2.973926e+09,2.973554e+09,2.973179e+09,2.973095e+09
Other Current Assets,2.345092e+09,2.131939e+09,2.193347e+09,2.656159e+09
Retained Earnings,5.710252e+10,5.138243e+10,4.422558e+10,3.250511e+10


In [54]:
dffinan2019 = pd.DataFrame(index=set100_key,columns=['TotalAssets','TotalCurrentLiabilities','TotalCurrentAssets','TotalLiab','TotalStockholderEquity'])

In [74]:
test_dffinan2019 = pd.read_excel('/content/dffinan2019.xlsx',index_col=0)

In [73]:
dffinan2019

,TotalAssets,TotalCurrentLiabilities,TotalCurrentAssets,TotalLiab,TotalStockholderEquity
ADVANC,289669121728.0,105255299816.0,47142151488.0,220274952106.0,69266099106.0
AOT,NaN,NaN,NaN,NaN,NaN
AWC,136493319649.0,11360686208.0,15937675970.0,45686923799.0,90806395850.0
BANPU,8068983000.0,1559530000.0,1362840000.0,5434068000.0,2028486000.0
BBL,3216743095000.0,2599473315000.0,136512845000.0,2788626786000.0,427751210000.0
...,...,...,...,...,...
TQM,4256958273.0,1815586774.0,3488872381.0,1906696776.0,2350260504.0
TTA,33473051000.0,3907535000.0,12313945000.0,10874489000.0,18187282000.0
VGI,NaN,NaN,NaN,NaN,NaN
WHA,82263830856.0,17837121215.0,17114310534.0,49163249675.0,29477631237.0


In [69]:
dffinan2019.to_excel('dffinan2019.xlsx')

In [67]:
year = '2019-12-31'

for x in  liststock: 
  getbal = pd.DataFrame(yf.Ticker(x).balancesheet)
  stockname = x.split('.')[0]
  try:
    dffinan2019['TotalStockholderEquity'][stockname] = float(getbal.loc['Total Stockholder Equity',year][0])
    print(stockname,'tt_curass = ',dffinan2019['TotalStockholderEquity'][stockname])

    dffinan2019['TotalLiab'][stockname] = float(getbal.loc['Total Liab',year][0])
    print(stockname,'tt_curass = ',dffinan2019['TotalLiab'][stockname])

    dffinan2019['TotalCurrentAssets'][stockname] = float(getbal.loc['Total Current Assets',year][0])
    print(stockname,'tt_curass = ',dffinan2019['TotalCurrentAssets'][stockname])

    dffinan2019['TotalCurrentLiabilities'][stockname] = float(getbal.loc['Total Current Liabilities',year][0])
    print(stockname,'tt_curlia = ',dffinan2019['TotalCurrentLiabilities'][stockname])   
  except:
    pass


ADVANC tt_curass =  69266099106.0
ADVANC tt_curass =  220274952106.0
ADVANC tt_curass =  47142151488.0
ADVANC tt_curlia =  105255299816.0
AWC tt_curass =  90806395850.0
AWC tt_curass =  45686923799.0
AWC tt_curass =  15937675970.0
AWC tt_curlia =  11360686208.0
BANPU tt_curass =  2028486000.0
BANPU tt_curass =  5434068000.0
BANPU tt_curass =  1362840000.0
BANPU tt_curlia =  1559530000.0
BBL tt_curass =  427751210000.0
BBL tt_curass =  2788626786000.0
BBL tt_curass =  136512845000.0
BBL tt_curlia =  2599473315000.0
BDMS tt_curass =  83813432690.0
BDMS tt_curass =  46480113870.0
BDMS tt_curass =  16325185436.0
BDMS tt_curlia =  19263149628.0
BEM tt_curass =  38780719689.0
BEM tt_curass =  72873588637.0
BEM tt_curass =  4447578321.0
BEM tt_curlia =  11428906824.0
BGRIM tt_curass =  28144399324.0
BGRIM tt_curass =  83015543806.0
BGRIM tt_curass =  33493108666.0
BGRIM tt_curlia =  25831745574.0
BH tt_curass =  19972155950.0
BH tt_curass =  5915901101.0
BH tt_curass =  11736441573.0
BH tt_cu

In [83]:
dffinan2020 = pd.DataFrame(index=set100_key,columns=['TotalAssets','TotalCurrentLiabilities','TotalCurrentAssets','TotalLiab','TotalStockholderEquity'])


In [89]:
dffinan2020

,TotalAssets,TotalCurrentLiabilities,TotalCurrentAssets,TotalLiab,TotalStockholderEquity
ACE,16429688248.0,1212730173.0,1841236407.0,4436855142.0,11992833106.0
ADVANC,350170577386.0,93813424234.0,42155398489.0,274481210049.0,75563983962.0
AEONTS,NaN,NaN,NaN,NaN,NaN
AMATA,40221654814.0,3909835436.0,10314911100.0,21469126770.0,15648208141.0
AOT,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
TTA,31029233000.0,5482515000.0,12323504000.0,11837814000.0,16153185000.0
TTB,1808331830000.0,1495565393000.0,258645928000.0,1603582105000.0,204712627000.0
TU,NaN,NaN,NaN,NaN,NaN
VGI,NaN,NaN,NaN,NaN,NaN


In [90]:
dffinan2020.to_excel('dffinan2020.xlsx')

In [85]:
year = '2020-12-31'

for x in  liststock: 
  getbal = pd.DataFrame(yf.Ticker(x).balancesheet)
  stockname = x.split('.')[0]
  try:
    dffinan2020['TotalStockholderEquity'][stockname] = float(getbal.loc['Total Stockholder Equity',year][0])
    print(stockname,'TotalStockholderEquity = ',dffinan2019['TotalStockholderEquity'][stockname])

    dffinan2020['TotalLiab'][stockname] = float(getbal.loc['Total Liab',year][0])
    print(stockname,'TotalLiab = ',dffinan2019['TotalLiab'][stockname])

    dffinan2020['TotalCurrentAssets'][stockname] = float(getbal.loc['Total Current Assets',year][0])
    print(stockname,'TotalCurrentAssets = ',dffinan2019['TotalCurrentAssets'][stockname])

    dffinan2020['TotalCurrentLiabilities'][stockname] = float(getbal.loc['Total Current Liabilities',year][0])
    print(stockname,'TotalCurrentLiabilities = ',dffinan2019['TotalCurrentLiabilities'][stockname])   
  except:
    pass

dffinan2020.sort_index(inplace=True)
dffinan2020.to_excel('dffinan2020.xlsx')

ADVANC TotalStockholderEquity =  69266099106.0
ADVANC TotalLiab =  220274952106.0
ADVANC TotalCurrentAssets =  47142151488.0
ADVANC TotalCurrentLiabilities =  105255299816.0
AWC TotalStockholderEquity =  90806395850.0
AWC TotalLiab =  45686923799.0
AWC TotalCurrentAssets =  15937675970.0
AWC TotalCurrentLiabilities =  11360686208.0
BANPU TotalStockholderEquity =  2028486000.0
BANPU TotalLiab =  5434068000.0
BANPU TotalCurrentAssets =  1362840000.0
BANPU TotalCurrentLiabilities =  1559530000.0
BBL TotalStockholderEquity =  427751210000.0
BBL TotalLiab =  2788626786000.0
BBL TotalCurrentAssets =  136512845000.0
BBL TotalCurrentLiabilities =  2599473315000.0
BDMS TotalStockholderEquity =  83813432690.0
BDMS TotalLiab =  46480113870.0
BDMS TotalCurrentAssets =  16325185436.0
BDMS TotalCurrentLiabilities =  19263149628.0
BEM TotalStockholderEquity =  38780719689.0
BEM TotalLiab =  72873588637.0
BEM TotalCurrentAssets =  4447578321.0
BEM TotalCurrentLiabilities =  11428906824.0
BGRIM TotalSt

In [91]:
dffinan2021 = pd.DataFrame(index=set100_key,columns=['TotalAssets','TotalCurrentLiabilities','TotalCurrentAssets','TotalLiab','TotalStockholderEquity'])

In [92]:
year = '2021-12-31'

for x in  liststock: 
  getbal = pd.DataFrame(yf.Ticker(x).balancesheet)
  stockname = x.split('.')[0]
  try:
    dffinan2021['TotalAssets'][stockname] = float(getbal.loc['Total Assets',year][0])
    print(stockname,'tt_asset = ',dffinan2021['TotalAssets'][stockname])

    dffinan2021['TotalStockholderEquity'][stockname] = float(getbal.loc['Total Stockholder Equity',year][0])
    print(stockname,'TotalStockholderEquity = ',dffinan2021['TotalStockholderEquity'][stockname])

    dffinan2021['TotalLiab'][stockname] = float(getbal.loc['Total Liab',year][0])
    print(stockname,'TotalLiab = ',dffinan2021['TotalLiab'][stockname])

    dffinan2021['TotalCurrentAssets'][stockname] = float(getbal.loc['Total Current Assets',year][0])
    print(stockname,'TotalCurrentAssets = ',dffinan2021['TotalCurrentAssets'][stockname])

    dffinan2021['TotalCurrentLiabilities'][stockname] = float(getbal.loc['Total Current Liabilities',year][0])
    print(stockname,'TotalCurrentLiabilities = ',dffinan2021['TotalCurrentLiabilities'][stockname])   
  except:
    pass

dffinan2021.sort_index(inplace=True)
dffinan2021.to_excel('dffinan2021.xlsx')

ADVANC tt_asset =  356221742939.0
ADVANC TotalStockholderEquity =  81698752332.0
ADVANC TotalLiab =  274397220562.0
ADVANC TotalCurrentAssets =  35566080608.0
ADVANC TotalCurrentLiabilities =  91868271839.0
AWC tt_asset =  138897289081.0
AWC TotalStockholderEquity =  79206620606.0
AWC TotalLiab =  59690668475.0
AWC TotalCurrentAssets =  905096152.0
AWC TotalCurrentLiabilities =  5667147579.0
BANPU tt_asset =  10945557000.0
BANPU TotalStockholderEquity =  2372157000.0
BANPU TotalLiab =  7842701000.0
BANPU TotalCurrentAssets =  2556459000.0
BANPU TotalCurrentLiabilities =  2908185000.0
BBL tt_asset =  4333280908000.0
BBL TotalStockholderEquity =  492727066000.0
BBL TotalLiab =  3838688351000.0
BBL TotalCurrentAssets =  135000007000.0
BBL TotalCurrentLiabilities =  3535224850000.0
BDMS tt_asset =  128453618430.0
BDMS TotalStockholderEquity =  83845192884.0
BDMS TotalLiab =  40688686254.0
BDMS TotalCurrentAssets =  24401394613.0
BDMS TotalCurrentLiabilities =  15861996067.0
BEM tt_asset = 

In [93]:
dffinan2021

,TotalAssets,TotalCurrentLiabilities,TotalCurrentAssets,TotalLiab,TotalStockholderEquity
ACE,18740200511.0,1327195065.0,2671920110.0,5371622899.0,13315330526.0
ADVANC,356221742939.0,91868271839.0,35566080608.0,274397220562.0,81698752332.0
AEONTS,NaN,NaN,NaN,NaN,NaN
AMATA,45470855090.0,8304171103.0,12443541130.0,23462422059.0,17723263521.0
AOT,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
TTA,38947081000.0,6618722000.0,18762055000.0,14546810000.0,21265661000.0
TTB,1759180734000.0,1448779401000.0,197792952000.0,1548345039000.0,210834555000.0
TU,NaN,NaN,NaN,NaN,NaN
VGI,NaN,NaN,NaN,NaN,NaN


In [ ]:
year = '2019-12-31'

for x in  liststock: 
  getbal = pd.DataFrame(yf.Ticker(x).balancesheet)
  stockname = x.split('.')[0]
  try:
    dffinan2019['TotalLiab'][stockname] = float(getbal.loc['Total Liab',year][0])
    print(stockname,'tt_curass = ',dffinan2019['TotalLiab'][stockname])
  except:
    dffinan2019['TotalLiab'][stockname] = np.nan
    print(stockname,'tt_curass= ไม่มี')

In [ ]:
year = '2019-12-31'

for x in  liststock: 
  getbal = pd.DataFrame(yf.Ticker(x).balancesheet)
  stockname = x.split('.')[0]
  try:
    dffinan2019['TotalCurrentAssets'][stockname] = float(getbal.loc['Total Current Assets',year][0])
    print(stockname,'tt_curass = ',dffinan2019['TotalCurrentAssets'][stockname])
  except:
    dffinan2019['TotalCurrentAssets'][stockname] = np.nan
    print(stockname,'tt_curass= ไม่มี')

In [ ]:
year = '2019-12-31'

for x in  liststock: 
  getbal = pd.DataFrame(yf.Ticker(x).balancesheet)
  stockname = x.split('.')[0]
  try:
    dffinan2019['TotalCurrentLiabilities'][stockname] = float(getbal.loc['Total Current Liabilities',year][0])
    print(stockname,'tt_curlia = ',dffinan2019['TotalCurrentLiabilities'][stockname])
  except:
    dffinan2019['TotalCurrentLiabilities'][stockname] = np.nan
    print(stockname,'tt_curlia = ไม่มี')


In [88]:
year = '2020-12-31'

for x in  liststock: 
  getbal = pd.DataFrame(yf.Ticker(x).balancesheet)
  stockname = x.split('.')[0]
  try:
    dffinan2020['TotalAssets'][stockname] = float(getbal.loc['Total Assets',year][0])
    print(stockname,'tt_asset = ',dffinan2020['TotalAssets'][stockname])
  except:
    dffinan2020['TotalAssets'][stockname] = np.nan
    print(stockname,'tt_asset = ไม่มี')


ADVANC tt_asset =  350170577386.0
AOT tt_asset = ไม่มี
AWC tt_asset =  133423011161.0
BANPU tt_asset =  9376660000.0
BBL tt_asset =  3822959685000.0
BDMS tt_asset =  136050251243.0
BEM tt_asset =  113615758299.0
BGRIM tt_asset =  130695879541.0
BH tt_asset =  24221665401.0
BLA tt_asset =  339778270000.0
BTS tt_asset = ไม่มี
CBG tt_asset =  17086676535.0
CPALL tt_asset =  523354329708.0
CPF tt_asset =  761719085000.0
CPN tt_asset =  221652288356.0
CRC tt_asset =  239179945394.0
DTAC tt_asset =  174280386848.0
EA tt_asset =  78483768062.0
EGCO tt_asset =  214437695000.0
GLOBAL tt_asset =  36020304871.0
GPSC tt_asset =  256656318380.0
GULF tt_asset =  245580506466.0
HMPRO tt_asset =  56090773174.0
INTUCH tt_asset =  52636607611.0
IRPC tt_asset =  173490552337.0
IVL tt_asset =  453171506000.0
JMART tt_asset =  25901089287.0
JMT tt_asset =  14270414754.0
KBANK tt_asset =  3658797759000.0
KCE tt_asset =  17377039415.0
KTB tt_asset =  3327780269000.0
KTC tt_asset =  91594496069.0
LH tt_asset 

In [58]:
year = '2019-12-31'

for x in  liststock: 
  getbal = pd.DataFrame(yf.Ticker(x).balancesheet)
  stockname = x.split('.')[0]
  try:
    dffinan2019['TotalAssets'][stockname] = float(getbal.loc['Total Assets',year][0])
    print(stockname,'tt_asset = ',dffinan2019['TotalAssets'][stockname])
  except:
    dffinan2019['TotalAssets'][stockname] = np.nan
    print(stockname,'tt_asset = ไม่มี')


ADVANC tt_asset =  289669121728.0
AOT tt_asset = ไม่มี
AWC tt_asset =  136493319649.0
BANPU tt_asset =  8068983000.0
BBL tt_asset =  3216743095000.0
BDMS tt_asset =  133661737019.0
BEM tt_asset =  111696769983.0
BGRIM tt_asset =  121692804686.0
BH tt_asset =  26181032487.0
BLA tt_asset =  333680074000.0
BTS tt_asset = ไม่มี
CBG tt_asset =  14780035767.0
CPALL tt_asset =  375617454248.0
CPF tt_asset =  634050559000.0
CPN tt_asset =  169933034161.0
CRC tt_asset =  183505495791.0
DTAC tt_asset =  167258148509.0
EA tt_asset =  70219694143.0
EGCO tt_asset =  208522604655.0
GLOBAL tt_asset =  35986387102.0
GPSC tt_asset =  252016615805.0
GULF tt_asset =  134277543428.0
HMPRO tt_asset =  52048956933.0
INTUCH tt_asset =  49534541988.0
IRPC tt_asset =  177850388896.0
IVL tt_asset =  380567907000.0
JMART tt_asset =  20372771133.0
JMT tt_asset =  10141669368.0
KBANK tt_asset =  3293888989000.0
KCE tt_asset =  16501002829.0
KTB tt_asset =  3012216120000.0
KTC tt_asset =  85409262233.0
LH tt_asset 

In [44]:
getbal = pd.DataFrame(yf.Ticker('BCH').balancesheet)
print(getbal.loc['Total Assets',year][0])

41273333000000.0


In [ ]:

  yf.Ticker(x).cashflow

In [133]:
data = yf.download(liststock, start="2019-01-01",end="2019-12-30")
dataclose2019 = data.Close
dataclose2019 = dataclose2019.dropna(axis=1)

[*********************100%***********************]  100 of 100 completed

9 Failed downloads:
- TIDLOR.BK: Data doesn't exist for startDate = 1546275600, endDate = 1577638800
- ONEE.BK: Data doesn't exist for startDate = 1546275600, endDate = 1577638800
- SCB.BK: Data doesn't exist for startDate = 1546275600, endDate = 1577638800
- STGT.BK: Data doesn't exist for startDate = 1546275600, endDate = 1577638800
- SCGP.BK: Data doesn't exist for startDate = 1546275600, endDate = 1577638800
- CRC.BK: Data doesn't exist for startDate = 1546275600, endDate = 1577638800
- TIPH.BK: Data doesn't exist for startDate = 1546275600, endDate = 1577638800
- OR.BK: Data doesn't exist for startDate = 1546275600, endDate = 1577638800
- KEX.BK: Data doesn't exist for startDate = 1546275600, endDate = 1577638800


Adj Close                                                          \
              ACE.BK   ADVANC.BK   AEONTS.BK   AMATA.BK     AOT.BK     AP.BK   
Date                                                                           
2019-01-02       NaN  150.048965  163.246170  19.066919  62.977455  5.005721   
2019-01-03       NaN  149.184128  166.368408  18.790588  62.242310  5.088459   
2019-01-04       NaN  153.075867  166.368408  19.711697  63.222504  5.088459   
2019-01-07       NaN  153.075867  166.814392  20.356472  62.977455  5.171199   
2019-01-08       NaN  153.508301  166.814392  20.356472  63.467552  5.253938   
...              ...         ...         ...        ...        ...       ...   
2019-12-23  4.079024  197.199112  173.134552  18.610271  74.565460  5.926332   
2019-12-24  4.000582  192.717300  172.220917  18.703791  72.825600  6.013484   
2019-12-25  3.980972  192.717300  172.220917  18.797308  73.571259  6.100636   
2019-12-26  4.000582  192.717300  172.220917  18.703791  74.316910  6.405667   
2019-12-27  4.020193  191.820923  173.134552  18.890827  73.819809  6.405667   

                                                        ...  Volume           \
              AWC.BK     BAM.BK   BANPU.BK      BBL.BK  ... TIPH.BK TISCO.BK   
Date                                                    ...                    
2019-01-02       NaN        NaN  11.952485  177.427383  ...     NaN  2929000   
2019-01-03       NaN        NaN  11.792048  179.175446  ...     NaN  2845000   
2019-01-04       NaN        NaN  12.594229  177.427383  ...     NaN  2321500   
2019-01-07       NaN        NaN  13.155755  181.797516  ...     NaN  3744100   
2019-01-08       NaN        NaN  13.075536  180.923492  ...     NaN  2266400   
...              ...        ...        ...         ...  ...     ...      ...   
2019-12-23  5.708539  16.219667   9.868606  145.075058  ...     NaN  1694900   
2019-12-24  5.758180  16.491508   9.952238  145.075058  ...     NaN   736500   
2019-12-25  5.758180  16.491508   9.868606  145.527008  ...     NaN   330800   
2019-12-26  5.758180  16.491508   9.952238  144.623138  ...     NaN  1028300   
2019-12-27  5.708539  16.400894   9.868606  144.171173  ...     NaN  1970500   

                                                                        \
              TOP.BK   TQM.BK    TRUE.BK   TTA.BK     TTB.BK     TU.BK   
Date                                                                     
2019-01-02   9628500  1443200   53716600   511000  106565295   4460900   
2019-01-03   6053600  4797800   62489300  1921000  109987431   9898300   
2019-01-04  15328200  2247200  144718300  1099300   97578182  13726000   
2019-01-07  15595400   848800   87312700  1366400  130708442  11431000   
2019-01-08   6634500   211800   48637500   846600   65327977   9121900   
...              ...      ...        ...      ...        ...       ...   
2019-12-23   8643200  3048000   87288200   373900  571444600  13850900   
2019-12-24   2593200  4122200   53762700  1008700  368798400   5583800   
2019-12-25   1550300  4485200   27132700  1256500  167424500   2213000   
2019-12-26   2282100  4176400   36008600  1091000  205011600   2810400   
2019-12-27   5251700  4911800  123747200   439600  189123200  11207100   

                                 
              VGI.BK     WHA.BK  
Date                             
2019-01-02  10150700   80663700  
2019-01-03  10202000   46101300  
2019-01-04   8319200  139231100  
2019-01-07  30698300   58618800  
2019-01-08   3764000   67614700  
...              ...        ...  
2019-12-23   5752600   94177100  
2019-12-24   5673800   95248400  
2019-12-25  13463300   36508500  
2019-12-26  31871400   61466000  
2019-12-27  15741100   50055200  

[243 rows x 600 columns]

In [153]:
data2 = yf.download(liststock, start="2020-01-01",end="2020-12-30")
dataclose2020 = data2.Close
dataclose2020 = dataclose2020.dropna(axis=1)

[*********************100%***********************]  100 of 100 completed

5 Failed downloads:
- TIDLOR.BK: Data doesn't exist for startDate = 1577811600, endDate = 1609261200
- ONEE.BK: Data doesn't exist for startDate = 1577811600, endDate = 1609261200
- SCB.BK: Data doesn't exist for startDate = 1577811600, endDate = 1609261200
- TIPH.BK: Data doesn't exist for startDate = 1577811600, endDate = 1609261200
- OR.BK: Data doesn't exist for startDate = 1577811600, endDate = 1609261200


In [2]:
data3 = yf.download(liststock, start="2021-01-01",end="2021-12-30")
dataclose2021 = data3.Close
dataclose2021 = dataclose2021.dropna(axis=1)
dataclose2021

NameError: ignored

In [ ]:
for l in dataclose2021:
  print(l  , ' ' , dataclose2020[l].isnull().sum())

In [ ]:
#2021 returntovol
for l in dataclose2021:
  print(l)
  vola2021 = dataclose2021[l].std(axis=0)
  returns2021 = (dataclose2021[l]['2021-12-29']/dataclose2021[l]['2021-01-04']-1) *100
  print('return = ',returns2021)
  print( 'returntovol = ', returns2021/vola2021)

In [ ]:
#2020 returntovol
for l in dataclose2020:
  print(l)
  vola2020 = dataclose2020[l].std(axis=0)
  returns2020 = (dataclose2020[l]['2020-12-29']/dataclose2020[l]['2020-01-02']-1) *100
  print('return = ',returns2020)
  print( 'returntovol = ', returns2020/vola2020)

In [ ]:
#2019 returntovol
for l in dataclose2019:
  print(l)
  vola2019 = dataclose2019[l].std(axis=0)
  returns2019 = (dataclose2019[l]['2019-12-27']/dataclose2019[l]['2019-01-02']-1) *100
  print(return2019)
  print( 'returntovol = ', returns2019/vola2019)

In [ ]:
dataclose2019.to_excel("close2019.xlsx")

In [161]:
dataclose2020.to_excel("close2020.xlsx")

In [172]:
dataclose2021.to_excel("close2021.xlsx")